In [1]:
import numpy as np 
import pandas as pd
import hyperopt #3
from hyperopt import hp, fmin, tpe, Trials
import optuna  #4
import sklearn
from sklearn import linear_model 
from sklearn import ensemble
from sklearn.model_selection import cross_val_score
from sklearn import metrics
from sklearn.model_selection import GridSearchCV #1
from sklearn.model_selection import RandomizedSearchCV #2
from sklearn.model_selection import train_test_split
from time import time
from sklearn.preprocessing import MinMaxScaler


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
data = pd.read_csv('data/_train_sem09')

In [10]:
data.head(5) #Проверим, правильно ли прочитался фаил

,"<!DOCTYPE html><html><head><meta name=""google"" content=""notranslate""><meta http-equiv=""X-UA-Compatible"" content=""IE=edge;""><style nonce=""VjnMlb7A1FshjE5dGsaJnQ"">@font-face{font-family:'Roboto';font-style:italic;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOkCnqEu92Fr1Mu51xMIzc.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:300;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmSU5fABc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:400;src:url(//fonts.gstatic.com/s/roboto/v18/KFOmCnqEu92Fr1Mu5mxP.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:500;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmEU9fABc9.ttf)format('truetype');}@font-face{font-family:'Roboto';font-style:normal;font-weight:700;src:url(//fonts.gstatic.com/s/roboto/v18/KFOlCnqEu92Fr1MmWUlfABc9.ttf)format('truetype');}</style><meta name=""referrer"" content=""origin""><title>hotels.csv - Google Диск</title><meta property=""og:title"" content=""hotels.csv""><meta property=""og:type"" content=""article""><meta property=""og:site_name"" content=""Google Docs""><meta property=""og:url"" content=""https://drive.google.com/file/d/1BEY_KCST7yeFyt8C6JC77eVCdhbZsnTN/view?usp=share_link&amp;usp=embed_facebook""><link rel=""shortcut icon"" href=""https://ssl.gstatic.com/images/branding/product/1x/drive_2020q4_32dp.png""><link rel=""stylesheet"" href=""https://fonts.googleapis.com/css?family=Google+Sans:300",400,500,"700"" nonce=""VjnMlb7A1FshjE5dGsaJnQ""><link rel=""stylesheet"" href=""https://www.gstatic.com/_/apps-fileview/_/ss/k=apps-fileview.v.ip7us8FVBMY.L.X.O/am=ABA/d=0/rs=AO0039t97Dr-0Ev5vk4XEF8IuW4FQD0lnA"" nonce=""VjnMlb7A1FshjE5dGsaJnQ""><script nonce=""qOc1Ii4jbZO7AdDIU-s38g"">_docs_flag_initialData={""docs-ails"":""docs_cold""","docs-fwds:""docs_sdf""","docs-crs:""docs_crs_nfd""",docs-l2t:0,docs-shdn:0,"docs-tfh:""""",info_params:{},...,0.82,0.83,1.57,0.84,0].8,[[null.2,null.202,null.203,"https://www.gstatic.com/og/_/js/k=og.qtm.en_US.5f4qC2U-w_g.es5.O/rt=j/m=qabr,q_dnp,qapid/exm=qaaw,qadd,qaid,qein,qhaw,qhba,qhbr,qhch,qhga,qhid,qhin,qhpr/d=1/ed=1/rs=AA2YrTv8jEr9OdvmSLBr0SHQ5df3rvNQIA]]]]",};this.gbar_=this.gbar_||{};(function(_){var window=this;
0,try{,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,/*,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Copyright The Closure Library Authors.,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,SPDX-License-Identifier: Apache-2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,*/,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
X, y = data.drop('Activity',axis = 1), data['Activity']
X_train, X_valid, y_train, y_valid = train_test_split(X, y, train_size = 0.8, random_state = 42)


In [5]:
display(X_train.shape)
display(X_valid.shape)

(3000, 1776)

(751, 1776)

In [6]:
lin_reg = linear_model.LogisticRegression(max_iter = 50, random_state = 42) 
rand_forest = ensemble.RandomForestClassifier(random_state = 42)

Первый способ подбора гиперпараметров

In [7]:
param_grid = [
             {
            'penalty': ['elasticnet','l1','l2', 'none'] ,#тип регурялизации
            'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'] #алгоритм оптимизаци
              }
             ]



param_grid = GridSearchCV(
  estimator = lin_reg,
  param_grid=param_grid, 
  cv = 10,
  n_jobs = -1,
  scoring= 'f1'
)

In [ ]:
%time param_grid.fit(X_train,y_train) #4 минуты

In [9]:
y_train_pred = param_grid.predict(X_train)
y_valid_pred = param_grid.predict(X_valid)

print(f'Наилучшие параметры: {param_grid.best_params_}') # penalty: 'l1', solver: saga
print(f'Метрика F1 на тренировойчной выборке: {metrics.f1_score(y_train,y_train_pred)}') #0.855
print(f'Метрика F1 на тестовой выборке: {metrics.f1_score(y_valid,y_valid_pred)}') #0.793

Наилучшие параметры: {'penalty': 'l1', 'solver': 'saga'}
Метрика F1 на тренировойчной выборке: 0.8551301684532925
Метрика F1 на тестовой выборке: 0.7931818181818182


In [30]:
param_grid_f = [
             {
            'max_depth': [5, 8, 10,12,14,16],
               'min_samples_leaf': [7, 8, 9,10]  
              }
             ]

 

param_grid = GridSearchCV(
  estimator = rand_forest,
  param_grid = param_grid_f, 
  cv = 10,
  n_jobs = -1,
  scoring= 'f1'
)

param_grid.fit(X_train,y_train) #1 минута 4 секунды

GridSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
             n_jobs=-1,
             param_grid=[{'max_depth': [5, 8, 10, 12, 14, 16],
                          'min_samples_leaf': [7, 8, 9, 10]}],
             scoring='f1')

In [31]:
y_train_pred = param_grid.predict(X_train)
y_valid_pred = param_grid.predict(X_valid)

print(f'Наилучшие параметры: {param_grid.best_params_}') # 'max_depth': 16, 'min_samples_leaf': 7
print(f'Метрика F1 на тренировойчной выборке: {metrics.f1_score(y_train,y_train_pred)}') #0.917
print(f'Метрика F1 на тестовой выборке: {metrics.f1_score(y_valid,y_valid_pred)}') #0.814

Наилучшие параметры: {'max_depth': 16, 'min_samples_leaf': 7}
Метрика F1 на тренировойчной выборке: 0.9175449473031618
Метрика F1 на тестовой выборке: 0.8137931034482758


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Второй способ подбора параметров

In [10]:
param_distributions = {
            'penalty': ['elasticnet','l1','l2', 'none'] ,#тип регурялизации
            'solver': ['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga'], #алгоритм оптимизаци
            'C': list(np.linspace(0.01, 1, 7, dtype=float)) 
              }
             


random_search = RandomizedSearchCV(
    estimator = lin_reg, 
    param_distributions = param_distributions, 
    cv = 10, 
    n_iter = 50, 
    n_jobs = -1,
    scoring = 'f1'
)  



In [ ]:
random_search.fit(X_train, y_train) #10 минут

In [12]:
y_train_pred_2 = random_search.predict(X_train)
y_valid_pred_2 = random_search.predict(X_valid)

print(f'Наилучшие параметры: {random_search.best_params_}') # 'solver': 'lbfgs', 'penalty': 'l2', 'C': 0.15142857142857144
print(f'Метрика F1 на тренировойчной выборке: {metrics.f1_score(y_train,y_train_pred_2)}') #0.857
print(f'Метрика F1 на тестовой выборке: {metrics.f1_score(y_valid,y_valid_pred_2)}') #0.793

Наилучшие параметры: {'solver': 'newton-cg', 'penalty': 'l2', 'C': 0.17500000000000002}
Метрика F1 на тренировойчной выборке: 0.8603076923076924
Метрика F1 на тестовой выборке: 0.7922814982973893


In [33]:
param_distributions_f = {'min_samples_leaf': list(np.linspace(5, 100, 50, dtype=int)),
              'max_depth': list(np.linspace(1, 30, 50, dtype=int)),
              'criterion':['entropy','gini']
              }
             


random_search = RandomizedSearchCV(
    estimator = rand_forest, 
    param_distributions = param_distributions_f, 
    cv = 10, 
    n_iter = 50, 
    n_jobs = -1,
    scoring = 'f1'
)  

random_search.fit(X_train, y_train)

RandomizedSearchCV(cv=10, estimator=RandomForestClassifier(random_state=42),
                   n_iter=50, n_jobs=-1,
                   param_distributions={'criterion': ['entropy', 'gini'],
                                        'max_depth': [1, 1, 2, 2, 3, 3, 4, 5, 5,
                                                      6, 6, 7, 8, 8, 9, 9, 10,
                                                      11, 11, 12, 12, 13, 14,
                                                      14, 15, 15, 16, 16, 17,
                                                      18, ...],
                                        'min_samples_leaf': [5, 6, 8, 10, 12,
                                                             14, 16, 18, 20, 22,
                                                             24, 26, 28, 30, 32,
                                                             34, 36, 37, 39, 41,
                                                             43, 45, 47, 49, 51,
                                                             53, 55, 57, 59, 61, ...]},
                   scoring='f1')

In [34]:
y_train_pred_2 = random_search.predict(X_train)
y_valid_pred_2 = random_search.predict(X_valid)

print(f'Наилучшие параметры: {random_search.best_params_}') # 'min_samples_leaf': 8, 'max_depth': 15, 'criterion': 'gini'
print(f'Метрика F1 на тренировойчной выборке: {metrics.f1_score(y_train,y_train_pred_2)}') #0.908
print(f'Метрика F1 на тестовой выборке: {metrics.f1_score(y_valid,y_valid_pred_2)}') #0.817

Наилучшие параметры: {'min_samples_leaf': 8, 'max_depth': 15, 'criterion': 'gini'}
Метрика F1 на тренировойчной выборке: 0.9080745341614906
Метрика F1 на тестовой выборке: 0.816933638443936


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


3 способ подбора параметров

In [13]:
space = {
    'penalty': hp.choice('penalty',['elasticnet','l1','l2']),
    'solver': hp.choice('solver',['lbfgs', 'liblinear', 'newton-cg', 'newton-cholesky', 'sag', 'saga']),
    'C': hp.quniform('C', 0.1, 1, 10)
}

In [14]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)

In [43]:

    
space_3 = {'penalty': hp.choice('penalty', ['elasticnet', 'none']),
              'solver': hp.choice('solver', ['saga']),
              'C': hp.uniform('C', 0.01, 1),
              'l1_ratio': hp.uniform('l1_ratio', 0, 1)} # параметр смешивания Elastic-Net 0 <= l1_ratio <= 1, обязателен, если penalty='elasticnet'  

random_state = 42

def hyperopt(space, cv=5, X=X_train, y=y_train, random_state=random_state):

    try:
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        l1_ratio=float(space['l1_ratio']),
        random_state=random_state,
        max_iter=50,
        n_jobs = -1, 
    )        
    except KeyError:
          
        model=linear_model.LogisticRegression(
        penalty = space['penalty'],
        solver = space['solver'],
        C=np.abs(float(space['C'])),
        random_state=random_state,
        max_iter=50,
        n_jobs = -1, 
           )
        
     
    
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()
  
    return -score





trials = Trials()

best = fmin(hyperopt, 
    algo = tpe.suggest,
    space = space_3, 
    max_evals= 50,
    trials = trials,
    rstate=np.random.RandomState(42),
    
  
)


100%|██████████| 50/50 [02:56<00:00,  3.54s/trial, best loss: -0.7861567128850439]


In [49]:
display(best['penalty'])
display(best['solver'])
display(best['C'])
display(best['l1_ratio'])

0

0

0.0684816920059412

0.12940876450080813

In [50]:


model = linear_model.LogisticRegression(
    random_state = random_state, 
    penalty = 'elasticnet',
    solver = 'saga',
    C = best['C'],
    l1_ratio = float(best['l1_ratio'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(f'F1 на тестовом наборе {metrics.f1_score(y_train, y_train_pred)}')
y_valid_pred = model.predict(X_valid)
print(f'F1 на тестовом наборе {metrics.f1_score(y_valid, y_valid_pred)}')

F1 на тестовом наборе 0.8341463414634146
F1 на тестовом наборе 0.8027366020524516


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\linear_model\_sag.py:350: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  warnings.warn(
c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but LogisticRegression was fitted without feature names
  warnings.warn(


In [35]:
# зададим пространство поиска гиперпараметров
space={'n_estimators': hp.quniform('n_estimators', 100, 200, 1),
       'max_depth' : hp.quniform('max_depth', 15, 26, 1),
       'min_samples_leaf': hp.quniform('min_samples_leaf', 2, 10, 1)
      }

random_state = 42
def hyperopt_rf(params, cv=5, X=X_train, y=y_train, random_state=random_state):
    # функция получает комбинацию гиперпараметров в "params"
    params = {'n_estimators': int(params['n_estimators']), 
              'max_depth': int(params['max_depth']), 
             'min_samples_leaf': int(params['min_samples_leaf'])
              }
  
    # используем эту комбинацию для построения модели
    model = ensemble.RandomForestClassifier(**params, random_state=random_state)

  
    model.fit(X, y)
    
  
    score = cross_val_score(model, X, y, cv=cv, scoring="f1", n_jobs=-1).mean()

    return -score

In [36]:

trials = Trials() # используется для логирования результатов

best=fmin(hyperopt_rf, # наша функция 
          space=space, # пространство гиперпараметров
          algo=tpe.suggest, # алгоритм оптимизации, установлен по умолчанию, задавать необязательно
          max_evals=20, # максимальное количество итераций
          trials=trials, # логирование результатов
          rstate=np.random.RandomState(random_state)# фиксируем для повторяемости результата
         )

100%|██████████| 20/20 [01:30<00:00,  4.51s/trial, best loss: -0.8085099484949732]


In [37]:
model = ensemble.RandomForestClassifier(
    random_state=random_state, 
    n_estimators=int(best['n_estimators']),
    max_depth=int(best['max_depth']),
    min_samples_leaf=int(best['min_samples_leaf'])
)
model.fit(X_train, y_train)
y_train_pred = model.predict(X_train)
print(f'F1 на тестовом наборе {metrics.f1_score(y_train, y_train_pred)}')
y_valid_pred = model.predict(X_valid)
print(f'F1 на тестовом наборе {metrics.f1_score(y_valid, y_valid_pred)}')

F1 на тестовом наборе 0.9903516962340492
F1 на тестовом наборе 0.8266360505166476


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(


Четыёртый способ

In [ ]:
#не знаю честно говоря почему не работает код, был бы очень рад узнать как это можно решить

def optuna_rf(trial):
  
  penalty = trial.suggest_categorical('penalty',['elasticnet', 'none'])
  solver = trial.suggest_categorical('solver',  ['saga'])
  C = trial.suggest_float('C',0.1, 1, 10)
  l1_ratio = trial.suggest_float('l1_ratio',0.1)

  model = linear_model.LogisticRegression( penalty =  penalty,
                                           solver= solver,
                                          C=C,
                                          l1_ratio = l1_ratio,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  #score = metrics.f1_score(y_train, model.predict(X_train))
  score = cross_val_score(model, X, y, cv=5, scoring="f1", n_jobs=-1).mean()
  return score


study = optuna.create_study(study_name="LogisticRegression", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)


# Trial.suggest_float() takes 4 positional arguments but 5 were given

Теперь проделаем точно то же самое, только для случайного леса


In [54]:
def optuna_rf(trial):
  # задаем пространства поиска гиперпараметров
  n_estimators = trial.suggest_int('n_estimators', 100, 200, 1)
  max_depth = trial.suggest_int('max_depth', 10, 30, 1)
  min_samples_leaf = trial.suggest_int('min_samples_leaf', 2, 10, 1)

  # создаем модель
  model = ensemble.RandomForestClassifier(n_estimators=n_estimators,
                                          max_depth=max_depth,
                                          min_samples_leaf=min_samples_leaf,
                                          random_state=random_state)
  # обучаем модель
  model.fit(X_train, y_train)
  
  score = cross_val_score(model, X_train, y_train, cv=5, scoring="f1", n_jobs=-1).mean()
  

  return score

In [ ]:
%%time
# cоздаем объект исследования
# можем напрямую указать, что нам необходимо максимизировать метрику direction="maximize"
study = optuna.create_study(study_name="RandomForestClassifier", direction="maximize")
# ищем лучшую комбинацию гиперпараметров n_trials раз
study.optimize(optuna_rf, n_trials=20)
# рассчитаем точность для тестовой выборки


In [53]:
model = ensemble.RandomForestClassifier(**study.best_params,random_state=random_state, )
model.fit(X_train, y_train)

y_train_pred = model.predict(X_train)
print(f'F1 на тестовом наборе {metrics.f1_score(y_train, y_train_pred)}')
y_valid_pred = model.predict(X_valid)
print(f'F1 на тестовом наборе {metrics.f1_score(y_valid, y_valid_pred)}')

F1 на тестовом наборе 0.9912881144990665
F1 на тестовом наборе 0.8279816513761467


c:\Users\Bigilittle\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\base.py:443: UserWarning: X has feature names, but RandomForestClassifier was fitted without feature names
  warnings.warn(
